In [1]:
import os
from langchain_groq import ChatGroq
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate
from dotenv import load_dotenv

In [2]:
load_dotenv() # Load environment variables from .env file

True

In [3]:
model = "llama3-8b-8192" # Model which is going to be used from Groq

In [4]:
llm = ChatGroq(model = model, temperature = 0.0) # Less creative, more factual responses
creative_llm = ChatGroq(model = model, temperature = 0.9) # More creative, less factual responses

In [5]:
# This is a sample article that we will use further.

article = """
Liverpool FC: The Renaissance Under Jürgen Klopp's Second Era

Liverpool Football Club is experiencing a remarkable transformation in what many are calling 
Jürgen Klopp's second era at the helm. After the departure of veteran players like Jordan 
Henderson, Roberto Firmino, and James Milner, the club has undergone a strategic rebuild 
that's breathing new life into the historic institution.

The midfield revolution has been particularly striking. The additions of Dominik Szoboszlai, 
Alexis Mac Allister, and Ryan Gravenberch have injected fresh energy into the team's core. 
These young talents, combined with the explosive potential of Darwin Núñez and the consistent 
brilliance of Mohamed Salah, have restored Liverpool's fearsome attacking reputation.

What's most impressive is how Klopp has managed this transition while maintaining the team's 
competitive edge. The Reds have seamlessly blended their renowned high-pressing style with 
a more nuanced possession game, adapting to the strengths of their new personnel. The 
emergence of academy graduates like Conor Bradley and Jarell Quansah also highlights the 
club's commitment to youth development.

However, challenges remain as the team seeks to reclaim their position at the summit of 
English football. The question of defensive stability and squad depth continues to spark 
debate among supporters. Yet, there's an undeniable sense of optimism at Anfield as this 
new-look Liverpool side shows all the hallmarks of another potentially dominant era in 
the club's storied history.
"""

In [6]:
# System prompt is a template that sets the context for the AI assistant.

system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant called {name} that helps generate article titles.", 
    input_variables = ["name"]
)

# User prompt is a template that asks (which is by the user) the AI assistant to generate a title based on the article content.
user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a name for a article.
The article is here for you to examine {article}

The name should be based of the context of the article.
Be creative, but make sure the names are clear, catchy,
and relevant to the theme of the article.

Only output the article name, no other explanation or
text can be provided.""",

    input_variables=["article"]
)

In [13]:
user_prompt.format(article = "TEST").content # This will print the formatted user prompt with the article content.

'You are tasked with creating a name for a article.\nThe article is here for you to examine TEST\n\nThe name should be based of the context of the article.\nBe creative, but make sure the names are clear, catchy,\nand relevant to the theme of the article.\n\nOnly output the article name, no other explanation or\ntext can be provided.'

In [8]:
first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt]) # Chat prompt template combines system and user prompts.

In [11]:
print(first_prompt.format(article = "TEST", name = "GROQ")) # This will merge both the system and user prompts into a single prompt.

System: You are an AI assistant called GROQ that helps generate article titles.
Human: You are tasked with creating a name for a article.
The article is here for you to examine TEST

The name should be based of the context of the article.
Be creative, but make sure the names are clear, catchy,
and relevant to the theme of the article.

Only output the article name, no other explanation or
text can be provided.


In [14]:
# Using LangChain Expression Language (LCEL) to construct our chain. 
# The inputs are defined in dictionary {} and the pipe operator (|) and the contents of the left side are fed to right side of the pipe.

chain_one = (
    {
        "article" : lambda x: x["article"], 
        "name" : lambda x: x["name"]
    }
    | first_prompt
    | creative_llm
    | {"article_title": lambda x: x.content}
)

In [16]:
# We will use invoke method to execute the chain with the article and name as inputs.

article_title = chain_one.invoke({
    "article": article, 
    "name" : "GROQ"
})

article_title

{'article_title': '"From Revolution to Renaissance: Liverpool FC\'s Bold Rebirth Under Jürgen Klopp"'}

### Multiple LLMChains to follow!

In [24]:
# Here we change the system prompt and a new user prompt.

system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that helps generate awesome articles."
)

second_user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a description for
the article. The article is here for you to examine:

---

{article}

---

Here is the article title '{article_title}'.

Output the SEO friendly article description. Also make sure dont exceed over 120 characters.
Do not output anything other than the description.""",
    input_variables=["article", "article_title"]
)

In [25]:
second_prompt = ChatPromptTemplate.from_messages(
    [
        system_prompt,
        second_user_prompt
    ]
)

In [26]:
chain_two = (
    {
        "article" : lambda x: x["article"],
        "article_title": lambda x: x["article_title"]
    }
    | second_prompt
    | llm
    | {"summary": lambda x: x.content}
)

# Here we have created a second chain.

In [27]:
article_desc_msg = chain_two.invoke(
    {
        "article": article,
        "article_title": article_title["article_title"]
    }
)

article_desc_msg   # This will invoke the second chain with the article and article title as inputs.
# The output will be the article description generated by the AI assistant.

{'summary': '"Jürgen Klopp\'s Liverpool FC undergoes a bold rebirth, blending youth and experience to reclaim their spot at the top of English football."'}